In [ ]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet('gen_data.parquet')

### Tính đầy đủ 

In [3]:
pd.set_option('display.max_rows', None)

Tỷ lệ missing theo cột

In [4]:
missing_col = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_pct': df.isna().mean() * 100
}).sort_values('missing_pct', ascending=False)

missing_col

,missing_count,missing_pct
COLLATERAL_VALUE,475839,19.826625
INCOME,120000,5.000000
TRINHDO,72000,3.000000
SOCIF,0,0.000000
CBAL_LONGTERM_LOAN,0,0.000000
OCCUPATION_TYPE,0,0.000000
CNT_DPD_30PLUS_6M,0,0.000000
HAS_LONGTERM_LOAN,0,0.000000
HAS_SHORTTERM_LOAN,0,0.000000
CBAL_SHORTTERM_LOAN,0,0.000000


In [5]:
pd.set_option('display.max_rows', 20)

Tỷ lệ missing theo row

In [6]:
df['ROW_MISSING_PCT'] = df.isna().mean(axis=1) * 100

df['ROW_MISSING_PCT'].describe()

count    2.400000e+06
mean     4.092151e-01
std      7.138921e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.470588e+00
max      4.411765e+00
Name: ROW_MISSING_PCT, dtype: float64

In [7]:
df['FLAG_HIGH_MISSING'] = (df['ROW_MISSING_PCT'] > 40).astype(int)

### Tính duy nhất 

Trùng khóa logic (SOCIF – year)

In [8]:
dup_key = df.duplicated(subset=['SOCIF', 'year'], keep=False)

df.loc[dup_key].shape

(0, 70)

Trùng toàn bộ bản ghi (record duplicate)

In [9]:
dup_full = df.duplicated(keep=False)

dup_full.sum()


np.int64(0)

In [10]:
dup_rate = dup_full.mean() * 100
print(f"Tỷ lệ bản ghi trùng lặp: {dup_rate:.2f}%")


Tỷ lệ bản ghi trùng lặp: 0.00%


### Tính kịp thời 

Check dùng future information (leakage)

In [11]:
# BAD_NEXT_12M chỉ được missing ở năm cuối
leak_check = df[
    (df['year'] < df['year'].max()) &
    (df['BAD_NEXT_12M'].isna())
]

leak_check.shape


(0, 70)

Check continuity theo năm

In [12]:
year_count = df.groupby('year')['SOCIF'].nunique()
year_count

year
2018    400000
2019    400000
2020    400000
2021    400000
2022    400000
2023    400000
Name: SOCIF, dtype: int64

### Tính phù hợp 

In [13]:
rule_valid_age = (
    df['TUOI'].between(0, 120)
)

df.loc[~rule_valid_age].shape


(0, 70)

In [14]:
rule_valid_ltv = df['LTV'].between(0, 400)

df.loc[~rule_valid_ltv].shape


(0, 70)

In [15]:
rule_valid_duration = (
    (df['DURATION_MAX'] >= 0) &
    (df['REMAINING_DURATION_MAX'] >= 0)
)

df.loc[~rule_valid_duration].shape


(0, 70)

In [16]:
valid_gender = df['C_GIOITINH'].isin(['M', 'F', 'O'])
df.loc[~valid_gender].shape

(0, 70)

In [17]:
df['FLAG_INVALID'] = ~(
    rule_valid_age &
    rule_valid_ltv &
    rule_valid_duration &
    valid_gender
)

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.describe()

,SOCIF,TRINHDO,SOHUUNHA,NHANVIENBIDV,INHERENT_RISK,REF_MONTH,REF_DAY,year,BASE_AUM,CURRENT_RISK,TUOI,SNAPSHOT_DATE,INCOME,CBAL,CBALORG,AFLIMT_MAX,AFLIMT_MIN,AFLIMT_AVG,CBAL_AVG,CBAL_MAX,CBAL_MIN,COLLATERAL_VALUE,LTV,N_AVG_DEPOSIT_12M,N_AVG_DEPOSIT_6M,N_AVG_DD_12M,N_AVG_CD_12M,FLAG_SALARY_ACC,FLAG_DEPOSIT,UTILIZATION_RATE,CNT_CREDIT_CARDS,AMT_CASH_ADVANCE_12M,FLAG_CASH_ADVANCE,PCT_PAYMENT_TO_BALANCE,CNT_MIN_PAY_6M,AVG_DAYS_PAST_DUE,DTI_RATIO,PTI_RATIO,MOB,CNT_OTHER_PRODUCTS,LIMIT_TO_INCOME,AMT_VAR_6M,CBAL_SHORTTERM_LOAN,CBAL_LONGTERM_LOAN,HAS_SHORTTERM_LOAN,HAS_LONGTERM_LOAN,CNT_DPD_30PLUS_6M,OCCUPATION_TYPE,DURATION_MAX,REMAINING_DURATION_MAX,TIME_TO_OP_MAX,RATE_AVG,PURCOD_MAX,PURCOD_MIN,MAX_DPD_12M,MAX_DPD_12M_OBS,AVG_OD_DPD_12M,SUM_ALL_OD_12M,BAD_CURRENT,XULYNO,MAX_NHOMNOCIC,N_AVG_OVERDUE_CBAL_12M,CBAL_TO_INC_12MON,REAL_GDP_GROWTH_12M,BAD_NEXT_12M,ROW_MISSING_PCT,FLAG_HIGH_MISSING
count,2.400000e+06,2.328000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2400000,2.280000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,1.924161e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2400000.0,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2.400000e+06,2400000.0
mean,1.200000e+06,1.855800e+00,3.382225e-01,1.029750e-02,7.010688e-17,6.498735e+00,1.452080e+01,2.020500e+03,1.040078e+07,1.665974e-01,3.720353e+01,2020-12-29 02:54:56.159998976,2.262860e+07,3.297642e+08,3.627548e+08,7.163083e+08,6.446775e+08,6.804929e+08,3.297157e+08,3.962120e+08,2.637725e+08,8.523221e+08,1.207265e+02,3.240266e+06,3.240266e+06,6.480527e+05,2.592213e+06,8.624429e-01,5.463367e-01,4.356772e-01,1.494581e+00,5.974919e+07,1.818250e-01,1.067599e+00,9.637629e-01,2.372091e+00,1.170814e+00,8.601475e-01,6.251819e+01,5.988854e-01,3.160412e+01,1.089114e-04,1.648788e+08,1.648854e+08,8.747633e-01,8.747633e-01,2.593213e-01,2.002271e+00,8.211065e+01,4.262277e+01,3.948788e+01,9.366556e+00,4.597413e+00,4.597413e+00,8.844162e+01,8.844162e+01,0.0,1.765570e+02,1.767304e-01,7.234750e-02,1.844410e+00,1.330231e+08,1.404976e+01,5.443333e+00,1.308383e-01,4.092151e-01,0.0
min,1.000000e+06,1.000000e+00,0.000000e+00,0.000000e+00,-1.414093e+00,1.000000e+00,1.000000e+00,2.018000e+03,1.976600e+04,-3.417804e+00,1.800000e+01,2018-01-01 00:00:00,8.036520e+06,0.000000e+00,0.000000e+00,2.070300e+08,1.863270e+08,1.966785e+08,0.000000e+00,0.000000e+00,0.000000e+00,2.117200e+04,0.000000e+00,7.400000e+01,7.400000e+01,1.400000e+01,6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,2.400000e+01,-1.031925e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,6.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2.580000e+00,0.000000e+00,0.000000e+00,0.0
25%,1.100000e+06,1.000000e+00,0.000000e+00,0.000000e+00,-7.346683e-01,4.000000e+00,7.750000e+00,2.019000e+03,2.352022e+06,-6.313609e-01,3.000000e+01,2019-06-28 00:00:00,1.673488e+07,1.623879e+08,1.778940e+08,5.042881e+08,4.538593e+08,4.790737e+08,1.593751e+08,1.916930e+08,1.275001e+08,3.001943e+06,1.430676e+01,4.265225e+05,4.265225e+05,8.530400e+04,3.412185e+05,1.000000e+00,0.000000e+00,2.690217e-01,1.000000e+00,0.000000e+00,0.000000e+00,8.233325e-01,0.000000e+00,0.000000e+00,6.758605e-01,4.

In [20]:
pd.set_option('display.max_columns', 20)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 71 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   SOCIF                   int64         
 1   C_GIOITINH              object        
 2   TRINHDO                 float64       
 3   TTHONNHAN               object        
 4   SOHUUNHA                int64         
 5   NHANVIENBIDV            int64         
 6   INHERENT_RISK           float64       
 7   REF_MONTH               int32         
 8   REF_DAY                 int32         
 9   year                    int64         
 10  BASE_AUM                int64         
 11  CURRENT_RISK            float64       
 12  TUOI                    int64         
 13  SNAPSHOT_DATE           datetime64[ns]
 14  INCOME                  float64       
 15  CBAL                    int64         
 16  CBALORG                 int64         
 17  AFLIMT_MAX              int64         
 18  AF

In [22]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns

for col in cat_cols:
    print(f"\n{'='*20} {col} {'='*20}")
    vc = df[col].value_counts(dropna=False)
    pct = df[col].value_counts(normalize=True, dropna=False) * 100
    print(pd.concat([vc, pct.rename('pct_%')], axis=1))


==================== C_GIOITINH ====================
              count     pct_%
C_GIOITINH                   
F           1176654  49.02725
M           1175634  48.98475
O             47712   1.98800

==================== TTHONNHAN ====================
             count   pct_%
TTHONNHAN                 
Single     1311000  54.625
Married    1089000  45.375

==================== SAMPLE_TYPE ====================
               count      pct_%
SAMPLE_TYPE                    
TRAIN        1600000  66.666667
OOS           400000  16.666667
OOT           400000  16.666667


### Tính chính xác

Tuổi khách hàng

In [23]:
rule_acc_age = df['TUOI'].between(18, 80)

LTV thực tế

In [24]:
rule_acc_ltv = df['LTV'] >= 0

Hạn mức – dư nợ

In [25]:
rule_acc_amount = (df['AFLIMT_MAX'] >= 0) & (df['CBAL'] >= 0) & (df['INCOME'] >= 0)

In [26]:
rule_acc_limit = df['AFLIMT_MAX'] >= 0 

Thu nhập – dư nợ

In [27]:
rule_acc_dti = df['DTI_RATIO'].between(0, 100)

CIC – DPD

In [28]:
rule_acc_cic = ~((df['MAX_DPD_12M_OBS'] >= 90) & (df['MAX_NHOMNOCIC'] < 3))

Flag & tỷ lệ chính xác

In [29]:
df['FLAG_INACCURATE'] = ~(
    rule_acc_age &
    rule_acc_ltv &
    rule_acc_limit &
    rule_acc_amount &
    rule_acc_dti &
    rule_acc_cic
)

accuracy_rate = 1 - df['FLAG_INACCURATE'].mean()
accuracy_rate

np.float64(0.95)

In [30]:
rule_acc_age.sum()

np.int64(2400000)

In [31]:
rule_acc_ltv.sum()

np.int64(2400000)

In [32]:
rule_acc_dti.sum()

np.int64(2400000)

In [33]:
rule_acc_cic.sum()

np.int64(2400000)

### Tính đồng nhất

CBAL vs cấu trúc kỳ hạn

In [34]:
import numpy as np
rule_cons_balance = np.isclose(
    df['CBAL'], 
    df['CBAL_SHORTTERM_LOAN'] + df['CBAL_LONGTERM_LOAN'], 
    atol=1.0 # Cho phép sai số 1 đơn vị tiền tệ
)

df.loc[~rule_cons_balance].shape

(0, 72)

Deposit breakdown

In [35]:
rule_cons_deposit = (
    abs(
        df['N_AVG_DEPOSIT_12M'] -
        (df['N_AVG_DD_12M'] + df['N_AVG_CD_12M'])
    ) <= 1
)
df.loc[~rule_cons_deposit].shape

(0, 72)

DPD logic

In [36]:
rule_cons_dpd = df['MAX_DPD_12M_OBS'] <= df['MAX_DPD_12M']

df.loc[~rule_cons_dpd].shape


(0, 72)

In [37]:
rule_struct_missing = ~(
    (df['MOB'] < 3) & (df['AMT_VAR_6M'].notna())
)

In [38]:
rule_flag_logic = ~(
    ((df['HAS_SHORTTERM_LOAN'] == 1) & (df['CBAL_SHORTTERM_LOAN'] <= 0)) |
    ((df['HAS_SHORTTERM_LOAN'] == 0) & (df['CBAL_SHORTTERM_LOAN'] > 0))
)

In [39]:
df['FLAG_OVERLIMIT'] = (df['CBAL'] > df['AFLIMT_MAX']).astype(int)
print(f"Số lượng vi phạm: {df['FLAG_OVERLIMIT'].sum()}")

Số lượng vi phạm: 0


Flag tổng hợp consistency

In [40]:
df['FLAG_INCONSISTENT'] = ~(
    rule_cons_balance &
    rule_cons_deposit &
    rule_cons_dpd &
    rule_struct_missing &
    rule_flag_logic
)

BẢNG TỔNG HỢP (RẤT NÊN CÓ)

In [41]:
quality_summary = pd.DataFrame({
    'Metric': [
        'Completeness',
        'Uniqueness',
        'Timeliness',
        'Validity',
        'Accuracy',
        'Consistency'
    ],
    'Issue_Rate_%': [
        df['FLAG_HIGH_MISSING'].mean() * 100,
        dup_full.mean() * 100,
        leak_check.shape[0] / len(df) * 100,
        df['FLAG_INVALID'].mean() * 100,
        df['FLAG_INACCURATE'].mean() * 100,
        df['FLAG_INCONSISTENT'].mean() * 100
    ]
})

quality_summary

,Metric,Issue_Rate_%
0,Completeness,0.0
1,Uniqueness,0.0
2,Timeliness,0.0
3,Validity,0.0
4,Accuracy,5.0
5,Consistency,0.0


In [42]:
df['FLAG_AGE_INVALID'] = (~rule_acc_age).astype(int)
df['FLAG_LTV_HIGH']    = (~rule_acc_ltv).astype(int)
df['FLAG_DTI_HIGH']    = (~rule_acc_dti).astype(int)
df['FLAG_CIC_LOW']     = (~rule_acc_cic).astype(int)
df['FLAG_INCONSISTENT_DEPOSIT'] = (~rule_cons_deposit).astype(int)

In [44]:
flag_cols = [c for c in df.columns if c.startswith('FLAG_')]

dq_flag_impact = []

for c in flag_cols:
    tmp = df.groupby(c)['BAD_NEXT_12M'].mean()
    dq_flag_impact.append({
        'FLAG': c,
        'BAD_RATE_0': tmp.get(0, np.nan),
        'BAD_RATE_1': tmp.get(1, np.nan),
        'LIFT': tmp.get(1, np.nan) / tmp.get(0, np.nan)
    })

dq_flag_impact = pd.DataFrame(dq_flag_impact).sort_values('LIFT', ascending=False)
dq_flag_impact

,FLAG,BAD_RATE_0,BAD_RATE_1,LIFT
2,FLAG_CASH_ADVANCE,0.106665,0.239615,2.246427
5,FLAG_INACCURATE,0.130827,0.131050,1.001703
0,FLAG_SALARY_ACC,0.269634,0.108701,0.403143
1,FLAG_DEPOSIT,0.246649,0.034673,0.140575
3,FLAG_HIGH_MISSING,0.130838,NaN,NaN
4,FLAG_INVALID,0.130838,NaN,NaN
6,FLAG_OVERLIMIT,0.130838,NaN,NaN
7,FLAG_INCONSISTENT,0.130838,NaN,NaN
8,FLAG_AGE_INVALID,0.130838,NaN,NaN
9,FLAG_LTV_HIGH,0.130838,NaN,NaN


In [45]:
for f in ['FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT']:
    print(f, df[f].value_counts(dropna=False))


FLAG_HIGH_MISSING FLAG_HIGH_MISSING
0    2400000
Name: count, dtype: int64
FLAG_INCONSISTENT FLAG_INCONSISTENT
False    2400000
Name: count, dtype: int64


In [46]:
dqa_cols_to_drop = [
    'FLAG_INVALID', 'FLAG_AGE_INVALID', 'FLAG_LTV_HIGH', 
    'FLAG_INCONSISTENT_DEPOSIT', 'FLAG_INCONSISTENT_DEPOSIT' 
    # Và các cột FLAG tạm khác nếu có
]
df.drop(columns=[c for c in dqa_cols_to_drop if c in df.columns], inplace=True)

In [47]:
print("7. Lọc bỏ hồ sơ rác & Đã xấu (Population Filter)...")

cond_remove = (
    (df['BAD_CURRENT'] == 1) |      # Loại người đang Bad
    (df['MAX_NHOMNOCIC'] >= 3) |    # Loại nhóm nợ xấu
    (df['XULYNO'] == 1) |           # Loại đã xử lý nợ
    (df['TUOI'] < 18) |   
    (df['TUOI'] > 80) |
    (df['DTI_RATIO'] > 100) |                            # Loại trẻ em
    (df['CBAL'] <= 100000) |             # Loại không có dư nợ (Inactive)
    (df['INCOME'] <= 0)
)

df = df[~cond_remove].copy()

7. Lọc bỏ hồ sơ rác & Đã xấu (Population Filter)...


In [48]:
df.drop(['FLAG_DTI_HIGH', 'FLAG_INACCURATE', 'FLAG_CIC_LOW', 'FLAG_HIGH_MISSING', 'FLAG_INCONSISTENT', 'ROW_MISSING_PCT', 'SOCIF', 'SNAPSHOT_DATE', 'REF_MONTH', 'REF_DAY', 'BAD_CURRENT', 'XULYNO', 'MAX_NHOMNOCIC', 'INHERENT_RISK', 'CURRENT_RISK', 'year', 'HAS_LONGTERM_LOAN', 'HAS_SHORTTERM_LOAN', 'FLAG_OVERLIMIT'], axis=1, inplace=True)

In [49]:
df.shape

(1740237, 56)

In [50]:
df.columns

Index(['C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA', 'NHANVIENBIDV',
       'BASE_AUM', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX',
       'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'UTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'AMT_CASH_ADVANCE_12M',
       'FLAG_CASH_ADVANCE', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M',
       'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'PTI_RATIO', 'MOB',
       'CNT_OTHER_PRODUCTS', 'LIMIT_TO_INCOME', 'AMT_VAR_6M',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M',
       'OCCUPATION_TYPE', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'BAD

In [51]:
target_col = 'BAD_NEXT_12M'
remove_cols = [target_col, 'SAMPLE_TYPE'] # Loại bỏ Target và nhãn chia tập khỏi X

# 2. Lọc dữ liệu theo từng tập
# TRAIN
mask_train = df['SAMPLE_TYPE'] == 'TRAIN'
train = df[mask_train].drop(columns='SAMPLE_TYPE')

# OOS
# mask_oos = df['SAMPLE_TYPE'] == 'OOS'
# X_oos = df[mask_oos].drop(columns=remove_cols)
# y_oos = df.loc[mask_oos, target_col]
mask_train = df['SAMPLE_TYPE'] == 'OOS'
oos = df[mask_train].drop(columns='SAMPLE_TYPE')

# OOT
# mask_oot = df['SAMPLE_TYPE'] == 'OOT'
# X_oot = df[mask_oot].drop(columns=remove_cols)
# y_oot = df.loc[mask_oot, target_col
mask_train = df['SAMPLE_TYPE'] == 'OOT'
oot = df[mask_train].drop(columns='SAMPLE_TYPE')

# 3. Kiểm tra kích thước (Shape Check) để đảm bảo không bị lệch dòng
print(f"TRAIN: {train.shape}")
print(f"OOS: {oos.shape}")
print(f"OOT: {oot.shape}")

# 4. Xuất ra file Parquet
# Lưu ý: X lưu dạng DataFrame, y lưu dạng Series (hoặc DataFrame 1 cột để an toàn index)
train.to_parquet('train.parquet', index=False)
oos.to_parquet('oos.parquet', index=False)
oot.to_parquet('oot.parquet', index=False)

# X_oos.to_parquet('X_oos.parquet', index=False)
# y_oos.to_frame().to_parquet('y_oos.parquet', index=False)

# X_oot.to_parquet('X_oot.parquet', index=False)
# y_oot.to_frame().to_parquet('y_oot.parquet', index=False)

TRAIN: (1137807, 55)
OOS: (300317, 55)
OOT: (302113, 55)
